In [3]:
import os, cv2 , sys
import pandas as pd
from pathlib import Path
import matplotlib.image as img
import numpy as np
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, f1_score


def prepare_data(dataset_dir):
    test_input_names = []
    test_output_names = []
    for file in os.listdir(dataset_dir):
        cwd = os.getcwd()
        test_input_names.append(os.path.join(str(dataset_dir),file))
    test_input_names.sort()
    return test_input_names


def filepath_to_name(full_name):
    file_name = os.path.basename(full_name)
    file_name = os.path.splitext(file_name)[0]
    return file_name

def load_image(path):
    image = cv2.cvtColor(cv2.imread(path, 1), cv2.COLOR_BGR2RGB)
    return image
def load_image1(path):
    image = cv2.cvtColor(cv2.imread(path, 1), cv2.COLOR_BGR2GRAY)
    return image

def compute_global_accuracy(label,pred):
    total = len(label)
    count = 0.0
    for i in range(total):
        if pred[i] == label[i]:
            count = count + 1.0
    return float(count) / float(total)


def evaluate_segmentation(pred, label, num_classes, score_averaging="weighted"):
    flat_pred = pred.flatten()
    flat_label = label.flatten()
    #flat_pred=np.append(flat_pred,[0,1])
    #flat_label=np.append(flat_label,[0,1])

    global_accuracy = compute_global_accuracy(flat_label, flat_pred)
    

    prec = precision_score(flat_label, flat_pred, average=score_averaging)
    #class_prec = precision_score(flat_label, flat_pred, labels=[0,1], average= None)
    class_prec = precision_score(flat_label, flat_pred, labels=[i for i in range(num_classes)], average= None)

    rec = recall_score(flat_label, flat_pred, average=score_averaging)
    #class_rec = recall_score(flat_label, flat_pred, labels=[0,1], average= None)
    class_rec = recall_score(flat_label, flat_pred, labels=[i for i in range(num_classes)], average= None)

    f1 = f1_score(flat_label, flat_pred, average=score_averaging)
    f1_wob = f1_score(flat_label, flat_pred, labels=[2], average='micro')

    #class_f1 = f1_score(flat_label, flat_pred, labels=[0,1], average= None)
    class_f1 = f1_score(flat_label, flat_pred, labels=[i for i in range(num_classes)], average= None)
    
    #Conf_Mat = confusion_matrix(flat_label, flat_pred, labels=[0,1])
    Conf_Mat = confusion_matrix(flat_label, flat_pred, labels=[i for i in range(num_classes)])

    return global_accuracy, prec, rec, f1, f1_wob, class_prec, class_rec, class_f1, Conf_Mat


def one_hot_it(label, label_values):
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis=-1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)
    return semantic_map


def reverse_one_hot(image):
    x = np.argmax(image, axis=-1)
    return x


def colour_code_segmentation(image, label_values):
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]
    return x

path = os.path.dirname(os.getcwd())
path = Path(path)


/home/chethan/.conda/envs/data/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#folder_name =  path/'Data/DataKeras/24_8_8/Predictions_after35Epoch'
#output_csv = path/'Data/DataKeras/24_8_8/Predictions_after35Epoch/pred_testData.csv'
folder_name = path/"testmodel/predtest/prediction"
output_csv = path/"fscore/fscore51.csv"
input_names = prepare_data(folder_name)
input_names


['/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0020_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0020_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Actinomyces.israeli_0021_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Actinomyces.israeli_0021_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/A

In [43]:
l=[]
lr=[]
input_names.sort()
b=pd.read_csv("/home/chethan/bac_clas/bacinfo.csv")
for i in range(int(len(input_names)/2)):
    # print(input_names[i*2])
    im=cv2.imread(input_names[i*2])
    im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    c,co=np.unique(im,return_counts=True)
    bc=c[:-1][np.argwhere(co[:-1]== co[:-1].max())[0][0]]
    
    g=np.array(b['grayscale']).round()


    if input_names[i*2][38:-16]==b['bacteria'][np.argwhere(g== bc)[0][0]]:
        print('kwj')
        l.append((input_names[i*2][38:-16],b['bacteria'][np.argwhere(g== bc)[0][0]]))
    else:
        if (input_names[i*2][38:-16],b['bacteria'][np.argwhere(g== bc)[0][0]]) not in lr:
             lr.append((input_names[i*2][38:-16],b['bacteria'][np.argwhere(g== bc)[0][0]]))



In [5]:
b=pd.read_csv("/home/chethan/bac_clas/bacinfo.csv")
list(b['bacteria']).index('Acinetobacter.baumanii')


0

In [58]:
input_names[0][53:-16]
input_names

['/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0020_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0020_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Actinomyces.israeli_0021_35pred.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/Actinomyces.israeli_0021_mask.png',
 '/home/chethan/bac_clas/testmodel/predtest/prediction/A

In [10]:
c=pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),'bacinfo.csv'))
grey_values=np.around(c['grayscale'])
s = 0
c=0
l=list(np.zeros(33))
ln=list(np.zeros(33))
Conf_Mat=0
for ind in range(int(len(input_names)/2)):
    bac=input_names[ind*2][53:-16]
    ind=list(b['bacteria']).index(bac)
    gt = load_image1(input_names[ind*2+1])
    print(input_names[ind*2+1])
    output_image = load_image1(input_names[ind*2])
    tr=np.sum(gt==output_image)
    #Conf_Mat = confusion_matrix(gt.flatten(), output_image.flatten(), labels=grey_values)
    a=tr/(1532*2048)
    l[ind]=l[ind]+a
    ln[ind]=ln[ind]+1
    s=s+a
    c=c+1
    #Sum=Sum+Conf_Mat


/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0017_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0018_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0019_mask.png
/home/chethan/bac_clas/testmodel/predtest/prediction/Acinetobacter.baumanii_0020_mask.png
/home/chet

In [11]:
s/c

0.949265553215706

In [7]:
a=[]
for i in range(len(l)):
    a.append(l[i]/ln[i])
for i in a :
    print(i)

0.9878627695935838
0.9888146583070146
0.9896409055019947
0.9758193645071476
0.9825988120221077
0.9939289498836437
0.9848165309175532
0.9906486348902925
0.9755817169838763
0.9769455930019947
0.9897730401221742
0.9912476235247673
0.7940102435172872
0.9937516882064494
0.9713806801653923
0.9884578623670213
0.986322930518617
0.983876329787234
0.9835185598819813
0.9890669153091756
0.9920108876329787
0.9868560141705452
0.9845902463223072
0.9576678986245013
0.9919352429978391
0.9959239553897938
0.9926329267785904
0.9959259033203125
0.8930517967711104
0.8731601796251662
0.8907434991065492
0.9873907210979056
0.8827956179355053


In [1]:
1504*2048

3080192

In [25]:
c=pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),'bacinfo.csv'))
ad=list(pd.DataFrame(c['bacteria'])['bacteria'])
# print(ad)
# csv=open(output_csv, 'w')
a=0
l=[]
for i in range(34):
    recall=Sum[i][i]/(sum(Sum[i,:]))
    prec=Sum[i][i]/(sum(Sum[:,i]))
    iou=Sum[i][i]/(sum(Sum[:,i])+sum(Sum[i,:])-Sum[i][i])
    acc=1-((sum(Sum[:,i])+sum(Sum[i,:])-2*Sum[i][i])/np.sum(Sum))
    f_score=2*prec*recall/(prec+recall)
    a=a+f_score
    # csv.write(f"{ad[i]},{recall},{prec},{f_score}\n")
    l.append(acc)

# print('average fscore',a/34)


In [41]:
Conf_Mat.diagonal()

array([      0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0, 1623416, 1294870])

In [37]:
per_class_accuracies

{192.0: 0.9999269526055519,
 29.0: 1.0,
 15.0: 0.999974676903258,
 150.0: 1.0,
 179.0: 0.9997892988489029,
 164.0: 1.0,
 75.0: 0.9988448772024601,
 104.0: 1.0,
 90.0: 1.0,
 76.0: 1.0,
 105.0: 1.0,
 91.0: 1.0,
 226.0: 1.0,
 151.0: 0.9994695135887633,
 180.0: 1.0,
 166.0: 1.0,
 38.0: 1.0,
 67.0: 1.0,
 53.0: 1.0,
 188.0: 1.0,
 217.0: 0.9995454828789894,
 203.0: 0.9999957794838763,
 113.0: 1.0,
 142.0: 1.0,
 128.0: 1.0,
 64.0: 1.0,
 78.0: 1.0,
 139.0: 0.9999165636427859,
 153.0: 1.0,
 102.0: 1.0,
 116.0: 0.9999983767245678,
 177.0: 1.0,
 0.0: 0.9475922280169548,
 255.0: 0.9498190372548205}

In [ ]:
t=0
tl=[]
for i in range(34):
    tp=Sum[i][i]
    tl.append(tp)
    a=a+f_score
print('average fscore',a/34)

In [ ]:
folder_name = path/"testmodel/predtest/35pred"
output_csv = path/"fscore/fscore46.csv"

input_names = prepare_data(folder_name)
input_names


In [18]:
c=pd.read_csv("D:\\bac_clas\\performance metrics\\metrics.ods")
c

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb7 in position 10: invalid start byte